In [1]:
import os

import dotenv

dotenv.load_dotenv()
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY", "")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN", "")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Multi-agent Collaboration"

In [2]:
from langchain_core.messages import BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


def create_agent(llm, tools, system_message: str):
    """Create an agent."""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards answering the question."
                " If you are unable to fully answer, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))

    return prompt | llm.bind_tools(tools)

In [3]:
from langchain_core.tools import tool


# create a tool
@tool
def get_the_secret_fact() -> str:
    """Returns the secret fact."""
    return "The secret fact is: A baby llama is called a 'Cria'."

In [4]:
import operator
from typing import Annotated, Sequence, TypedDict


# This defines the object that is passed between each node
# in the graph. We will create different nodes for each agent and tool
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str

In [5]:
# import functools

# from langchain_core.messages import AIMessage
# from langchain_huggingface import HuggingFaceEndpoint


# # Helper function to create a node for a given agent
# def agent_node(state, agent, name):
#     result = agent.invoke(state)
#     # We convert the agent output into a format that is suitable to append to the global state
#     if isinstance(result, ToolMessage):
#         pass
#     else:
#         result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
#     return {
#         "messages": [result],
#         # Since we have a strict workflow, we can
#         # track the sender so we know who to pass to next.
#         "sender": name,
#     }


# llm = HuggingFaceEndpoint(
#     endpoint_url="meta-llama/Meta-Llama-3-8B-Instruct", max_new_tokens=3000
# )

# # Research agent and node
# research_agent = create_agent(
#     llm,
#     [get_the_secret_fact],
#     system_message="You should provide accurate data for the chart_generator to use.",
# )
# research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

# # chart_generator
# chart_agent = create_agent(
#     llm,
#     [],
#     system_message="Any charts you display will be visible by the user.",
# )
# chart_node = functools.partial(agent_node, agent=chart_agent, name="chart_generator")